# Pipeline de Visualização de Resultados RAGAS

**Carregamento e Processamento dos Dados**
   - Para cada modelo de embedding:
     - Monta o caminho do arquivo CSV correspondente.
     - Verifica se o arquivo existe.
     - Lê o arquivo CSV com pandas.
     - Para cada métrica:
       - Filtra as linhas do CSV que correspondem à métrica (ex: `faithfulness_1`, `faithfulness_2`, ...).
       - Converte a coluna `Pontuação` para valores numéricos.
       - Calcula a média dos valores encontrados e armazena no dicionário `results`.

**Exibição dos Resultados**
   - Imprime no console as médias de cada métrica para cada modelo de embedding.

**Resumo:**  
O pipeline automatiza a leitura dos resultados de avaliação de diferentes modelos de embedding, calcula as médias das métricas relevantes e exibe esses resultados de forma organizada, facilitando a comparação de desempenho entre os modelos.

Os resultados foram representados em graficos de radar, com pequenos ajustes para facilitar a visualização.

In [36]:
# Parte 1: Imports, carregamento de dados e cálculo das médias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# Definir o caminho base (ajustando para a pasta RAGAS na raiz)
base_path = "01_Questions_JSON/gemma-2b/"

# Lista de modelos de embeddings
models = [
    "all-MiniLM-L6-v2",
    "all-mpnet-base-v2",
    "paraphrase-multilingual-MiniLM-L12-v2",
    "distiluse-base-multilingual-cased-v2",
    "stsb-xlm-r-multilingual",
    "neuralmind-bert-base-portuguese-cased"
]

# Lista de métricas
metrics = ["faithfulness", "answer_relevancy", "context_recall", "context_precision"]

# Dicionário para armazenar os resultados médios
results = {model: {metric: 0 for metric in metrics} for model in models}

# Carregar e processar os dados
for model in models:
    # Construir o caminho para o arquivo CSV
    file_path = os.path.join(base_path, model, f"Answers_{model}_ragas_results.csv")
    
    # Verificar se o arquivo existe
    if not os.path.exists(file_path):
        print(f"Arquivo não encontrado: {file_path}")
   
    try:
        # Carregar o CSV
        df = pd.read_csv(file_path)
        
        # Calcular a média para cada métrica
        for metric in metrics:
            # Filtrar linhas que começam com a métrica
            metric_rows = df[df['Métrica'].str.startswith(metric + "_")].copy()
            if not metric_rows.empty:
                # Converter para numero e calcular a média
                metric_rows['Pontuação'] = pd.to_numeric(metric_rows['Pontuação'], errors='coerce')
                results[model][metric] = metric_rows['Pontuação'].mean()
            else:
                print(f"Métrica {metric} não encontrada para o modelo {model}")
    except Exception as e:
        print(f"Erro ao processar {file_path}: {e}")

print("Resultados calculados com sucesso!")
print("\nValores médios por modelo e métrica:")
for model in models:
    print(f"\n{model}:")
    for metric in metrics:
        print(f"  {metric}: {results[model][metric]:.4f}")

Resultados calculados com sucesso!

Valores médios por modelo e métrica:

all-MiniLM-L6-v2:
  faithfulness: 0.8571
  answer_relevancy: 0.5784
  context_recall: 0.7143
  context_precision: 0.7143

all-mpnet-base-v2:
  faithfulness: 0.9286
  answer_relevancy: 0.4540
  context_recall: 0.5714
  context_precision: 0.4643

paraphrase-multilingual-MiniLM-L12-v2:
  faithfulness: 0.7143
  answer_relevancy: 0.5834
  context_recall: 1.0000
  context_precision: 0.6714

distiluse-base-multilingual-cased-v2:
  faithfulness: 0.7143
  answer_relevancy: 0.4359
  context_recall: 0.8571
  context_precision: 0.6238

stsb-xlm-r-multilingual:
  faithfulness: 0.7143
  answer_relevancy: 0.5358
  context_recall: 0.7143
  context_precision: 0.5714

neuralmind-bert-base-portuguese-cased:
  faithfulness: 0.9286
  answer_relevancy: 0.4503
  context_recall: 0.5714
  context_precision: 0.4286
